## Data from File 5.3 Suppliers Bank Account Assignments

### Step 1. Load required package

In [1]:
import pandas as pd
import pandasql as ps
from pathlib import Path

### Step 2 - Get sample from source file

In [2]:
sourceFolder = '/content/drive/Shareddrives/Esperanto/Supports/Suppliers/5_Suppliers_Payees_BankAccounts'
workFolder = '/content/drive/Shareddrives/Esperanto/Supports/Suppliers/Work/5_Bank_Accounts'
fileName = '5.3. SUPPLIER_BANK_ACCOUNT_ASSIGNMENTS_COPPEL.csv'
inputTextFile = open(Path(sourceFolder, fileName), 'r', encoding='latin-1')
if inputTextFile:
    numTextLines = 0
    while True:
        numTextLines += 1
        textLine = inputTextFile.readline()
        print(textLine)
        if numTextLines >10:
            break
    inputTextFile.close()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Shareddrives/Esperanto/Supports/Suppliers/5_Suppliers_Payees_BankAccounts/5.3. SUPPLIER_BANK_ACCOUNT_ASSIGNMENTS.csv'